In [1]:
!git clone https://github.com/huggingface/search-and-learn

Cloning into 'search-and-learn'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 235 (delta 48), reused 43 (delta 43), pack-reused 160 (from 1)
Receiving objects: 100% (235/235), 87.99 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [2]:
%cd search-and-learn
!pip install -e '.[dev]'

/content/search-and-learn
Obtaining file:///content/search-and-learn
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 91.7 MB/s eta 0:00:00
   ━━━

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Cookbook` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Cookbook`


In [ ]:
#%cd search-and-learn
#!python scripts/test_time_compute.py recipes/Llama-3.2-1B-Instruct/best_of_n.yaml

In [ ]:
#%cd search-and-learn
#!python scripts/test_time_compute.py recipes/Llama-3.2-1B-Instruct/dvts.yaml

Make a recipe for building an LLM system with scaling test-time compute:

* Use the same code as in the recipes but with the smallest approach, thinking about an LLM chat bot system instead of a benchmark
* LLM class where a user talks and the LLM response after thinking for a while.
* Add the "thinking for x seconds message"
* Final function that only generates the final output based on the input, without the rest of the intermediate messages.
* Can this be combined with agents/VLMs...?
* Space in HF



In [2]:
import torch
from vllm import LLM

from sal.config import Config
from sal.models.reward_models import RLHFFlow
from sal.search import beam_search, best_of_n, dvts
from sal.utils.data import get_dataset, save_dataset
from sal.utils.score import score

APPROACHES = {
    "beam_search": beam_search,
    "dvts": dvts,
    "best_of_n": best_of_n,
}

config = Config
print(Config.approach)
print(config)

approach_fn = APPROACHES['best_of_n']

model_path="meta-llama/Llama-3.2-1B-Instruct"
prm_path="RLHFlow/Llama3.1-8B-PRM-Deepseek-Data"
gpu_memory_utilization = (0.5)
seed=42

llm = LLM(
    model=model_path,
    gpu_memory_utilization=gpu_memory_utilization,
    enable_prefix_caching=True,
    seed=seed,
)

prm = RLHFFlow(prm_path)

/usr/local/lib/python3.10/dist-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


best_of_n
<class 'sal.config.Config'>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

WARNING 12-24 16:16:27 arg_utils.py:953] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-24 16:16:27 config.py:1005] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-24 16:16:27 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, qu

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [3]:
from datasets import Dataset, load_dataset

config.n=32
config.sort_completed= True
config.filter_duplicates= True

search_batch_size=25
dataset_start= 0
dataset_end= 1

dataset_name="HuggingFaceH4/MATH-500"
dataset_split='test'

dataset = load_dataset(dataset_name, split=dataset_split)
dataset = dataset.select(range(dataset_start, dataset_end))

README.md:   0%|          | 0.00/412 [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/447k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 1
})

In [5]:
dataset[0]

{'problem': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$',
 'solution': 'We have that $r = \\sqrt{0^2 + 3^2} = 3.$  Also, if we draw the line connecting the origin and $(0,3),$ this line makes an angle of $\\frac{\\pi}{2}$ with the positive $x$-axis.\n\n[asy]\nunitsize(0.8 cm);\n\ndraw((-0.5,0)--(3.5,0));\ndraw((0,-0.5)--(0,3.5));\ndraw(arc((0,0),3,0,90),red,Arrow(6));\n\ndot((0,3), red);\nlabel("$(0,3)$", (0,3), W);\ndot((3,0), red);\n[/asy]\n\nTherefore, the polar coordinates are $\\boxed{\\left( 3, \\frac{\\pi}{2} \\right)}.$',
 'answer': '\\left( 3, \\frac{\\pi}{2} \\right)',
 'subject': 'Precalculus',
 'level': 2,
 'unique_id': 'test/precalculus/807.json'}

In [ ]:
#dataset = get_dataset(config)
'''
dataset = dataset.map(
    #approach_fn,
    best_of_n,
    batched=True,
    batch_size=search_batch_size,
    fn_kwargs={"config": config, "llm": llm, "prm": prm},
    desc="Running search",
    load_from_cache_file=False,
)
'''

In [ ]:
batch = {
    "problem": [dataset[0]["problem"]],
    #"solution": [dataset[0]["solution"]],
    #"answer": [dataset[0]["answer"]],
    #"subject": [dataset[0]["subject"]],
    #"level": [dataset[0]["level"]],
    #"unique_id": [dataset[0]["unique_id"]],
}

In [19]:
result = best_of_n(x=batch, config=config, llm=llm, prm=prm)

In [20]:
result = Dataset.from_dict(result)

In [21]:
result = score(result, config)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions:   0%|          | 0/6 [00:00<?, ?it/s]

Subsample 1:   0%|          | 0/1 [00:00<?, ? examples/s]

Extract answers 1:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute weighted pred 1:   0%|          | 0/1 [00:00<?, ? examples/s]

...

Extract answers 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute weighted pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute majority pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Compute naive pred 32:   0%|          | 0/1 [00:00<?, ? examples/s]

Computing majority & weighted predictions: 100%|██████████| 6/6 [00:00<00:00,  6.45it/s]


In [22]:
result['pred'][0]

'<|start_header_id|>assistant<|end_header_id|>\n\n## Step 1: Recall the conversion formulas\nTo convert from rectangular (Cartesian) coordinates $(x,y)$ to polar coordinates $(r,\\theta)$, we use the following formulas:\n$r = \\sqrt{x^2 + y^2}$ and $\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right)$.\n\n## Step 2: Substitute the values\nGiven the point $(0,3)$ in rectangular coordinates, we can substitute $x = 0$ and $y = 3$ into the formulas:\n$r = \\sqrt{0^2 + 3^2} = \\sqrt{0 + 9} = \\sqrt{9} = 3$\n\n## Step 3: Calculate $\\theta$\nNow, we can calculate $\\theta = \\tan^{-1}\\left(\\frac{3}{0}\\right)$. However, since division by zero is undefined, and the point $(0,3)$ is on the positive y-axis, the angle $\\theta$ will be $\\frac{\\pi}{2}$ radians.\n\n## Step 4: Write the polar coordinates\nThe polar coordinates of $(0,3)$ are $(r, \\theta) = \\left(3, \\frac{\\pi}{2}\\right)$.\n\nThe final answer is: $\\boxed{\\left(3, \\frac{\\pi}{2}\\right)}$'

In [23]:
formatted_output = result['pred'][0].replace("<|start_header_id|>assistant<|end_header_id|>\n\n", "").strip()
formatted_output

'## Step 1: Recall the conversion formulas\nTo convert from rectangular (Cartesian) coordinates $(x,y)$ to polar coordinates $(r,\\theta)$, we use the following formulas:\n$r = \\sqrt{x^2 + y^2}$ and $\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right)$.\n\n## Step 2: Substitute the values\nGiven the point $(0,3)$ in rectangular coordinates, we can substitute $x = 0$ and $y = 3$ into the formulas:\n$r = \\sqrt{0^2 + 3^2} = \\sqrt{0 + 9} = \\sqrt{9} = 3$\n\n## Step 3: Calculate $\\theta$\nNow, we can calculate $\\theta = \\tan^{-1}\\left(\\frac{3}{0}\\right)$. However, since division by zero is undefined, and the point $(0,3)$ is on the positive y-axis, the angle $\\theta$ will be $\\frac{\\pi}{2}$ radians.\n\n## Step 4: Write the polar coordinates\nThe polar coordinates of $(0,3)$ are $(r, \\theta) = \\left(3, \\frac{\\pi}{2}\\right)$.\n\nThe final answer is: $\\boxed{\\left(3, \\frac{\\pi}{2}\\right)}$'

In [24]:
from IPython.display import display, Markdown, Latex
display(Markdown(formatted_output))

## Step 1: Recall the conversion formulas
To convert from rectangular (Cartesian) coordinates $(x,y)$ to polar coordinates $(r,\theta)$, we use the following formulas:
$r = \sqrt{x^2 + y^2}$ and $\theta = \tan^{-1}\left(\frac{y}{x}\right)$.

## Step 2: Substitute the values
Given the point $(0,3)$ in rectangular coordinates, we can substitute $x = 0$ and $y = 3$ into the formulas:
$r = \sqrt{0^2 + 3^2} = \sqrt{0 + 9} = \sqrt{9} = 3$

## Step 3: Calculate $\theta$
Now, we can calculate $\theta = \tan^{-1}\left(\frac{3}{0}\right)$. However, since division by zero is undefined, and the point $(0,3)$ is on the positive y-axis, the angle $\theta$ will be $\frac{\pi}{2}$ radians.

## Step 4: Write the polar coordinates
The polar coordinates of $(0,3)$ are $(r, \theta) = \left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

In [67]:
import gc
from vllm.distributed.parallel_state import destroy_model_parallel, destroy_distributed_environment

destroy_model_parallel()
destroy_distributed_environment()
del llm.llm_engine.model_executor
del llm
gc.collect()
torch.cuda.empty_cache()
print("Successfully delete the llm pipeline and free the GPU memory!")

Processed prompts:   0%|          | 0/4106 [01:56<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Successfully delete the llm pipeline and free the GPU memory!


In [68]:
import gc
import time


def clear_memory():
    # Delete variables if they exist in the current global scope
    if "llm" in globals():
        del globals()["llm"]
    if "dataset" in globals():
      del globals()["dataset"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

GPU allocated memory: 3.07 GB
GPU reserved memory: 3.16 GB
